# Splunk App for Data Science and Deep Learning - Notebook for Autoencoder with TensorFlow and Keras (version 2.15)

## Autoencoder Example
This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Splunk App for Data Science and Deep Learning (DSDL). As an example we use a custom autoencoder built on keras and tensorflow.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# mltkc_import
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

2024-04-25 17:27:27.210161: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)

numpy version: 1.26.4
pandas version: 2.2.1
TensorFlow version: 2.16.1
Keras version: 3.2.0


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup iris.csv <br>| fit MLTKContainer algo=autoencoder mode=stage epochs=100 batch_size=4 components=2 petal_length petal_width sepal_length sepal_width into app:iris_autoencoder

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("my_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# mltkc_stage
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [4]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("iris_autoencoder")
print(df[0:1])
print(df.shape)
print(str(param))

   sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
(150, 4)
{'options': {'params': {'algo': 'autoencoder', 'mode': 'stage', 'epochs': '100', 'batch_size': '4', 'components': '2'}, 'args': ['petal_length', 'petal_width', 'sepal_length', 'sepal_width'], 'feature_variables': ['petal_length', 'petal_width', 'sepal_length', 'sepal_width'], 'model_name': 'iris_autoencoder', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '10000', 'max_distinct_cat_values_for_classifiers': '10000', 'max_distinct_cat_values_for_scoring': '10000', 'max_fit_time': '6000', 'max_inputs': '10000000', 'max_memory_usage_mb': '16000', 'max_model_size_mb': '3000', 'max_score_time': '6000', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['petal_length', 'petal_width', 'sepal_length', 'sepal_width']}


## Stage 2 - create and initialize a model

In [5]:
# mltkc_init
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    X = df[param['feature_variables']]
    print("FIT build model with input shape " + str(X.shape))
    components = 3
    activation_fn = 'relu'
    # learning_rate = 0.001
    # epsilon=0.00001 # default 1e-07
    if 'options' in param:
        if 'params' in param['options']:
            if 'components' in param['options']['params']:
                components = int(param['options']['params']['components'])
            if 'activation_func' in param['options']['params']:
                activation_fn = param['options']['params']['activation_func']
    input_shape = int(X.shape[1])
    encoder_layer = keras.layers.Dense(components, input_dim=input_shape, activation=activation_fn, kernel_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None), bias_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None))
    decoder_layer = keras.layers.Dense(input_shape, activation=activation_fn, kernel_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None), bias_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None))
    model = keras.Sequential()
    model.add(encoder_layer)
    model.add(decoder_layer)
    #opt = keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
# test mltkc_stage_create_model
model = init(df,param)
print(model.summary())

FIT build model with input shape (150, 4)


/usr/local/lib/python3.9/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22 (88.00 B)

 Trainable params: 22 (88.00 B)

 Non-trainable params: 0 (0.00 B)

None


## Stage 3 - fit the model

In [7]:
# mltkc_stage_create_model_fit
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    X = df[param['feature_variables']]
    model_epochs = 100
    model_batch_size = 32
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model_epochs = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
    # connect model training to tensorboard
    log_dir="/srv/notebooks/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    # run the training
    returns['fit_history'] = model.fit(x=X,
                                       y=X, 
                                       verbose=2, 
                                       epochs=model_epochs, 
                                       batch_size=model_batch_size, 
                                       #validation_data=(X, Y),
                                       callbacks=[tensorboard_callback])
    # memorize parameters
    returns['model_epochs'] = model_epochs
    returns['model_batch_size'] = model_batch_size
    returns['model_loss_acc'] = model.evaluate(x = X, y = X)
    return returns

In [8]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

Epoch 1/100
38/38 - 1s - 26ms/step - accuracy: 0.0000e+00 - loss: 46.5630
Epoch 2/100
38/38 - 0s - 4ms/step - accuracy: 0.0000e+00 - loss: 33.0022
Epoch 3/100
38/38 - 0s - 3ms/step - accuracy: 0.0000e+00 - loss: 23.8413
Epoch 4/100
38/38 - 0s - 3ms/step - accuracy: 0.0000e+00 - loss: 17.6326
Epoch 5/100
38/38 - 0s - 3ms/step - accuracy: 0.0000e+00 - loss: 13.3247
Epoch 6/100
38/38 - 0s - 3ms/step - accuracy: 0.0000e+00 - loss: 10.3247
Epoch 7/100
38/38 - 0s - 3ms/step - accuracy: 0.0067 - loss: 8.1804
Epoch 8/100
38/38 - 0s - 3ms/step - accuracy: 0.2800 - loss: 6.6330
Epoch 9/100
38/38 - 0s - 3ms/step - accuracy: 0.7867 - loss: 5.4928
Epoch 10/100
38/38 - 0s - 3ms/step - accuracy: 0.9600 - loss: 4.6150
Epoch 11/100
38/38 - 0s - 3ms/step - accuracy: 1.0000 - loss: 3.9303
Epoch 12/100
38/38 - 0s - 3ms/step - accuracy: 1.0000 - loss: 3.3970
Epoch 13/100
38/38 - 0s - 4ms/step - accuracy: 1.0000 - loss: 2.9542
Epoch 14/100
38/38 - 0s - 3ms/step - accuracy: 1.0000 - loss: 2.6008
Epoch 15/100

In [14]:
dir(model)
model.inputs

[<KerasTensor shape=(None, 4), dtype=float32, sparse=None, name=keras_tensor>]

## Stage 4 - apply the model

In [15]:
# mltkc_stage_create_model_apply
def apply(model,df,param):
    X = df[param['feature_variables']]
    reconstruction = model.predict(x = X)
    intermediate_layer_model = keras.Model(inputs=model.inputs, outputs=model.layers[0].output)
    hidden = intermediate_layer_model.predict(x = X)
    y_hat = pd.concat([pd.DataFrame(reconstruction).add_prefix("reconstruction_"), pd.DataFrame(hidden).add_prefix("hidden_")], axis=1)
    return y_hat

In [16]:
# test mltkc_stage_create_model_apply
y_hat = apply(model,df,param)
print(y_hat)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
     reconstruction_0  reconstruction_1  reconstruction_2  reconstruction_3  \
0            2.120456          0.785697          4.074873          2.693462   
1            2.107639          0.722653          4.012266          2.558842   
2            1.998948          0.690182          3.908722          2.539497   
3            2.100163          0.692428          3.981321          2.494955   
4            2.095052          0.775607          4.048545          2.683353   
..                ...               ...               ...               ...   
145          5.101612          1.720695          6.952622          3.323231   
146          4.850245          1.555095          6.636131          3.076226   
147          4.999534          1.654109          6.824662          3.224406   
148          5.095782          1.688707          6.921326          3.254595   
149          4.780374          1.521764          6.5589

## Stage 5 - save the model

In [11]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # save keras model to hdf5 file
    # https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models
    model.save(MODEL_DIRECTORY + name + ".keras")
    return model

In [12]:
save(model,"autoencoder_iris_autoencoder")

## Stage 6 - load the model

In [13]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = keras.models.load_model(MODEL_DIRECTORY + name + ".keras")
    return model

In [14]:
model2 = load("autoencoder_iris_autoencoder")
model2

## Stage 7 - provide a summary of the model

In [15]:
# return model summary
def summary(model=None):
    returns = {"version": {"tensorflow": tf.__version__, "keras": keras.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        model.summary(print_fn=lambda x: s.append(x+'\n'))
        returns["summary"] = ''.join(s)
    return returns

In [16]:
summary(model)

{'version': {'tensorflow': '2.15.0', 'keras': '2.15.0'},
 'summary': 'Model: "sequential"\n_________________________________________________________________\n Layer (type)                Output Shape              Param #   \n=================================================================\n dense (Dense)               (None, 2)                 10        \n                                                                 \n dense_1 (Dense)             (None, 4)                 12        \n                                                                 \n=================================================================\nTotal params: 22 (88.00 Byte)\nTrainable params: 22 (88.00 Byte)\nNon-trainable params: 0 (0.00 Byte)\n_________________________________________________________________\n'}

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code